# Validation croisée
Une meilleure façon de tester vos modèles.

## Introduction
L'apprentissage automatique est un **processus itératif**.

Vous serez confronté à des choix : quelles **variables prédictives** utiliser, quels types de modèles privilégier, quels arguments fournir à ces modèles, etc. Jusqu'à présent, vous avez pris ces décisions de manière guidée par les données, en mesurant la qualité des modèles grâce à un ensemble de validation (ou ensemble de test).

Cependant, cette méthode a quelques inconvénients. Imaginez que vous avez un jeu de données avec 5000 lignes. Habituellement, vous réservez environ 20 % des données comme ensemble de validation, soit 1000 lignes. Cela laisse une part de hasard dans la détermination des scores des modèles. Par exemple, un modèle pourrait bien fonctionner sur un ensemble donné de 1000 lignes, même s'il serait peu performant sur un autre ensemble de 1000 lignes.

Dans un cas extrême, imaginez que l'ensemble de validation ne contienne qu'une seule ligne. Si vous comparez des modèles, déterminer lequel prédit le mieux sur une seule donnée relèverait presque du hasard !

En général, **plus l'ensemble de validation est grand, moins il y a d'aléa (ou "bruit") dans votre mesure de qualité des modèles, et plus celle-ci est fiable**. 

Malheureusement, avoir un grand ensemble de validation signifie réduire la taille des données d'entraînement, ce qui entraîne souvent des modèles moins performants.



## Qu'est-ce que la validation croisée ?
La validation croisée consiste à exécuter votre processus de modélisation sur différents sous-ensembles des données pour obtenir plusieurs mesures de la qualité des modèles.

Prenons un exemple : on divise les données en 5 parties, chacune représentant 20 % de l'ensemble. Ces parties sont appelées des ***"folds"***.

**Exemple : Division en 5 folds**
- Expérience 1 : Le premier fold est utilisé comme ensemble de validation, tandis que les autres servent à l'entraînement.
- Expérience 2 : Le deuxième fold devient l'ensemble de validation, et les autres servent à l'entraînement.
- Et ainsi de suite, jusqu'à ce que chaque fold ait été utilisé une fois comme ensemble de validation.

Ainsi, chaque ligne des données est utilisée à un moment donné comme ensemble de validation, offrant une mesure de qualité basée sur l'ensemble complet des données (même si toutes les lignes ne sont pas utilisées simultanément).



### Quand utiliser la validation croisée ?
La validation croisée fournit une mesure plus précise de la qualité des modèles. Cela est particulièrement important si vous prenez de nombreuses décisions concernant les modèles. Cependant, elle peut être plus longue à exécuter, car elle implique de tester plusieurs modèles (un pour chaque fold).

**Recommandations** :
- Pour les petits jeux de données, utilisez la validation croisée : la charge computationnelle supplémentaire n'est pas un problème.
- Pour les grands jeux de données, un seul ensemble de validation est souvent suffisant, car les modèles s'entraînent plus vite et les données sont assez nombreuses pour éviter le besoin de réutiliser certaines pour le test.

---

**Exemple : Validation croisée avec Scikit-learn est fait directement sur PIPELINES**

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor

# Charger les données
melbourne_file_path = 'melb_data.csv'
melbourne_data = pd.read_csv(melbourne_file_path)

# Supprimer les lignes avec une cible manquante et séparer la cible des prédicteurs
melbourne_data.dropna(axis=0, subset=['Price'], inplace=True)
y = melbourne_data['Price']
X = melbourne_data.drop(['Price'], axis=1)

# Identifier les colonnes numériques et catégoriques
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

# Variables catégorielles à conserver
keep_cols = ['Type', 'Method', 'Regionname']

# Supprimer les variables catégoriques inutilisées
categorical_cols = [col for col in categorical_cols if col in keep_cols]
X = X[numerical_cols + categorical_cols]

# Diviser en données d'entraînement et de validation
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

# Afficher les colonnes restantes
print("Colonnes numériques :", numerical_cols)
print("Colonnes catégoriques :", categorical_cols)
print("Colonnes utilisées dans X_train :", X_train.columns)


Colonnes numériques : ['Rooms', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'YearBuilt', 'Lattitude', 'Longtitude', 'Propertycount']
Colonnes catégoriques : ['Type', 'Method', 'Regionname']
Colonnes utilisées dans X_train : Index(['Rooms', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'Lattitude', 'Longtitude',
       'Propertycount', 'Type', 'Method', 'Regionname'],
      dtype='object')


In [15]:
print(X.dtypes)

Rooms              int64
Distance         float64
Postcode         float64
Bedroom2         float64
Bathroom         float64
Car              float64
Landsize         float64
BuildingArea     float64
YearBuilt        float64
Lattitude        float64
Longtitude       float64
Propertycount    float64
Type              object
Method            object
Regionname        object
dtype: object


In [17]:
X = X.select_dtypes(include=['number'])  # Garde uniquement les colonnes numériques

In [19]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Identification des colonnes numériques et catégoriques
numeric_features = X.select_dtypes(include=['number']).columns
categorical_features = X.select_dtypes(exclude=['number']).columns

# Transformer pour les colonnes numériques et catégoriques
preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='mean'), numeric_features),  # Imputation pour colonnes numériques
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)  # Encodage pour colonnes catégoriques
    ]
)

# Nouveau pipeline
my_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(n_estimators=50, random_state=0))
])


In [21]:
# Relancer la validation croisée avec le pipeline mis à jour 
scores = -1 * cross_val_score(my_pipeline, X, y, cv=5, scoring='neg_mean_absolute_error')
print("Scores MAE :\n", scores)
print("Score MAE moyen :", scores.mean())


Scores MAE :
 [215843.83846588 204063.10068764 193213.5288239  162618.89111473
 166564.13952767]
Score MAE moyen : 188460.6997239638


---

### Conclusion
La validation croisée offre une meilleure estimation de la qualité des modèles. De plus, elle simplifie votre code : vous n'avez plus besoin de gérer des ensembles de validation et d'entraînement séparés. Pour les petits jeux de données, c'est une amélioration précieuse.